In [1]:

import pandas as pd
import numpy as np
from validation.check_eventids import check_eventids, check_extension_eventids
from validation.check_fields   import check_fields
from validation.check_scientifiname import check_scientificname
import taxa
import requests
import io
NaN = np.nan
from utils import removesuffix

In [2]:
dataf = pd.read_csv('sepaq_bd_1944-2020.csv')
dataf = dataf.rename(columns=str.lower)
scientitifname_list = dataf['scientificname'].unique()
dataf.head()


,datasetid,occurrenceid,eventdate,nomstation,location,decimallongitude,decimallatitude,samplingprotocol,scientificname,scientificnameid,taxonid,occurrencestatus,organismquantitytype,organismquantity
0,AQ-RIPE,AQ-RIPE-1944-1,1944-09-10T07:30:00,Pêche Arthur Matte,Neuville,-71.622365,46.693963,Pêche à fascine commerciale,Acipenser fulvescens,urn:lsid:marinespecies.org:taxname:271694,https://www.gbif.org/fr/species/2402120,present,Nombre d'individus,2.0
1,AQ-RIPE,AQ-RIPE-1944-2,1944-09-10T07:30:00,Pêche Arthur Matte,Neuville,-71.622365,46.693963,Pêche à fascine commerciale,Acipenser oxyrinchus,urn:lsid:marinespecies.org:taxname:151802,https://www.gbif.org/fr/species/4287132,present,Nombre d'individus,3.0
2,AQ-RIPE,AQ-RIPE-1944-3,1944-09-10T07:30:00,Pêche Arthur Matte,Neuville,-71.622365,46.693963,Pêche à fascine commerciale,Anguilla rostrata,urn:lsid:marinespecies.org:taxname:158562,https://www.gbif.org/fr/species/5212956,present,Nombre d'individus,3.0
3,AQ-RIPE,AQ-RIPE-1944-4,1944-09-10T07:30:00,Pêche Arthur Matte,Neuville,-71.622365,46.693963,Pêche à fascine commerciale,Catostomidae sp.,urn:lsid:marinespecies.org:taxname:154179,https://www.gbif.org/fr/species/3384,present,Nombre d'individus,5.0
4,AQ-RIPE,AQ-RIPE-1944-5,1944-09-10T07:30:00,Pêche Arthur Matte,Neuville,-71.622365,46.693963,Pêche à fascine commerciale,Coregonus clupeaformis,urn:lsid:marinespecies.org:taxname:158726,https://www.gbif.org/fr/species/2351119,present,Nombre d'individus,1.0


In [3]:
def remove_suffix(name: str) -> str:
    suffixes = [
        ' sp.',
        ' spp.',
        ' sp',
        ' spp',
    ]
    for suf in suffixes:
        print(suf)
        if name.endswith(suf):
            return removesuffix(name, suf)

    # If no suffix was found return the original name
    # with whitespace removed
    return name.strip()

def function_suffix_removal(data_valid_nom_scientifique):
    liste_noms_pre_modif    = []
    liste_noms              = []
    liste_noms_sans_suffix  = []
    liste_noms_sp           = []
    liste_noms_sp_point     = []
    liste_noms_spp          = []
    liste_noms_spp_point    = []
    	
    liste_noms_pre_modif = data_valid_nom_scientifique['scientificname'].to_list()

    for rows in data_valid_nom_scientifique.index:
        scientificName = data_valid_nom_scientifique['scientificname'].iloc[rows].replace(" ","%20")
        
        if scientificName[-3:] != 'sp.':
            if scientificName[-2:] != 'sp':
                if scientificName[-3:] != 'spp':
                    if scientificName[-4:] != 'spp.':
                        liste_noms_sans_suffix.append(scientificName)

        if scientificName[-3:] == 'sp.':
            scientificName = scientificName[:-6]
            liste_noms_sp_point.append(scientificName)
        if scientificName[-2:] == 'sp': 
            scientificName = scientificName[:-5]
            liste_noms_sp.append(scientificName)
        if scientificName[-4:] == 'spp.':
            scientificName = scientificName[:-7]
            liste_noms_spp_point.append(scientificName)
        if scientificName[-3:] == 'spp':
            scientificName = scientificName[:-6]
            liste_noms_spp.append(scientificName)
        
        liste_noms.append(scientificName)

    liste_noms = list(set(liste_noms))

    return liste_noms_pre_modif, liste_noms, liste_noms_sans_suffix, liste_noms_sp, liste_noms_sp_point, liste_noms_spp, liste_noms_spp_point

def function_add_suffix(nom, liste_noms_sans_suffix, liste_noms_sp, liste_noms_sp_point, liste_noms_spp, liste_noms_spp_point):
    list_of_list = {}
    nom_recomposed = nom
    nom_recomposed = nom_recomposed.replace("%20", " ")
    nom_recomposed_sp = NaN
    nom_recomposed_sp_point = NaN
    nom_recomposed_spp = NaN
    nom_recomposed_spp_point = NaN

    if nom in liste_noms_sp:
        nom_recomposed_sp = nom_recomposed + " sp"
        list_of_list["noms_sp"] = nom_recomposed_sp

    if nom in liste_noms_sp_point:
        nom_recomposed_sp_point = nom_recomposed + " sp."
        list_of_list["noms_sp_point"] = nom_recomposed_sp_point

    if nom in liste_noms_spp:
        nom_recomposed_spp = nom_recomposed + " spp"
        list_of_list["noms_spp"] = nom_recomposed_spp

    if nom in liste_noms_spp_point:
        nom_recomposed_spp_point = nom_recomposed + " spp."
        list_of_list["noms_spp_point"] = nom_recomposed_spp_point

    if nom in liste_noms_sans_suffix:
        list_of_list["noms_sans_suffix"] = nom_recomposed


    return list_of_list  


In [4]:
check_scientificname(dataf, 'names')

RuntimeError: asyncio.run() cannot be called from a running event loop